In [ ]:
import pandas as pd         # 导入pandas库，它就像一个超级强大的Excel工具，专门用来处理表格数据（DataFrame）。

# 加载数据集
data = pd.read_csv('健康咨询客户数据集.csv')
# 使用pandas的read_csv函数，读取名为 '健康咨询客户数据集.csv' 的CSV文件。
# 将文件内容加载到一个名为 'data' 的DataFrame（可以理解为表格）中。

# 查看表结构基本信息
print(data.head())          # 打印出DataFrame的前5行。这能帮助我们快速查看数据的基本结构、列名和前几条记录，初步了解数据长什么样。

# 显示每一列的空缺值数量
print(data.isnull().sum())  # `data.isnull()` 会创建一个与原DataFrame形状相同，但内容为True/False的DataFrame，
                            # 其中True表示原始位置有缺失值（NaN）。
                            # `.sum()` 会对这个True/False DataFrame进行求和，因为True被算作1，False被算作0，
                            # 所以结果就是每一列缺失值的总数量。这有助于我们了解缺失值的分布情况。

# 删除含有缺失值的行
data_cleaned = data.dropna()
# 使用DataFrame的 .dropna() 方法，删除所有包含任何缺失值（NaN）的行。
# 这样做是为了确保我们处理的数据是完整的，避免缺失值对后续分析造成影响。
# 将处理后的结果赋值给 `data_cleaned`，保留原始 `data` 不变。

# 转换 'Your age' 列的数据类型为整数类型，并处理异常值
data_cleaned.loc[:, 'Your age'] = pd.to_numeric(data_cleaned['Your age'], errors='coerce')
# `data_cleaned.loc[:, 'Your age']`：使用 `loc` 索引器，同时指定行和列，安全地修改DataFrame的副本。
# `pd.to_numeric(...)`：尝试将 'Your age' 列转换为数值类型。
# `errors='coerce'`：这个参数非常重要。如果遇到无法转换成数字的值（例如，文本字符串），它不会引发错误，而是将其替换为 `NaN`（缺失值）。
data_cleaned = data_cleaned.dropna(subset=['Your age'])
# 由于上一步可能产生了新的 `NaN` 值（由 `errors='coerce'` 引起），这里再次使用 `dropna()`。
# `subset=['Your age']`：表示只在 'Your age' 这一列中查找并删除含有 `NaN` 值的行。
data_cleaned = data_cleaned[data_cleaned['Your age'] >= 0]
# 筛选数据，只保留 'Your age' 大于或等于0的行。这可以排除负数年龄等不合理的记录。
data_cleaned.loc[:, 'Your age'] = data_cleaned['Your age'].astype(int)
# 再次使用 `loc` 安全地将 'Your age' 列的数据类型转换为整数（int）。

print(data_cleaned['Your age'].dtype) # 打印 'Your age' 列的最终数据类型，确认它已成功转换为整数。

# 检查和删除重复值
duplicates_removed = data_cleaned.duplicated().sum()
# `data_cleaned.duplicated()` 会返回一个布尔系列（True/False），其中True表示该行是与之前某行完全重复的。
# `.sum()` 会对布尔系列求和，计算出重复行的总数量。
data_cleaned = data_cleaned.drop_duplicates()
# 使用DataFrame的 .drop_duplicates() 方法，删除 `data_cleaned` 中所有完全相同的重复行。

print(f"Removed {duplicates_removed} duplicate rows") # 打印出总共删除了多少重复行。

from sklearn.preprocessing import LabelEncoder # 从scikit-learn库中导入LabelEncoder，这是一个用于将类别标签编码为数字的工具。

# 归一化 'How do you describe your current level of fitness ?' 列
label_encoder = LabelEncoder()      # 创建一个LabelEncoder的实例（一个“标签编码器”）。
data_cleaned['How do you describe your current level of fitness ?'] = label_encoder.fit_transform(data_cleaned['How do you describe your current level of fitness ?'])
# data_cleaned['How do you describe your current level of fitness ?']：选取需要进行标签编码的列。
# `label_encoder.fit_transform(...)`：这是LabelEncoder的核心方法。它做了两件事：
#   1. `fit`：学习该列中所有独特的类别（例如，“高”, “中”, “低”）。
#   2. `transform`：将每个类别替换为一个唯一的整数编码（例如，高->0, 中->1, 低->2）。
# 这是一种将非数值的类别数据转换为数值形式的方法，便于机器学习模型处理。

print(data_cleaned['How do you describe your current level of fitness ?'].unique())
# 打印出 'How do you describe your current level of fitness ?' 列中所有独特的（编码后的）数值。
# 这可以验证编码是否成功，并了解编码后的数字范围。

from sklearn.preprocessing import LabelEncoder # 再次导入LabelEncoder（虽然上面已经导入过，但这里再次出现可能是在分步骤编写代码时的一个习惯）。
import matplotlib.pyplot as plt                # 导入matplotlib.pyplot库，主要用于创建静态、动态交互式的可视化图表。

# 去掉列名中的空格
data.columns = data.columns.str.strip()
# `data.columns` 获取所有列名。
# `.str.strip()` 对每个列名字符串应用 `strip()` 方法，移除字符串开头和结尾的空白字符。
# 这样做是为了避免列名中不必要的空格导致后续访问列时出错。
# 将处理后的列名重新赋值回 `data.columns`。

# 显示数据集的列名
print(data.columns)                 # 打印更新后的所有列名，确认空格是否已成功去除。

# 删除包含缺失值的行
data_cleaned = data.dropna(subset=['How often do you exercise?'])
# 这一次，我们创建 `data_cleaned` 时，专门针对 'How often do you exercise?' 这一列进行缺失值删除。
# `subset=['How often do you exercise?']` 表示只删除在该列中有缺失值的行，而不是删除任何列有缺失值的行。

# 统计不同健身频率的分布情况
exercise_frequency_counts = data_cleaned['How often do you exercise?'].value_counts()
# `data_cleaned['How often do you exercise?'].value_counts()`：统计 'How often do you exercise?' 列中每个独特值出现的次数。
# 结果是一个Series，索引是健身频率类别，值是对应的计数。

# 绘制饼图
plt.figure(figsize=(10, 6))         # 创建一个新的图形窗口，并设置其大小为10英寸宽、6英寸高。
exercise_frequency_counts.plot.pie(autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors)
# 使用Series的内置 `plot.pie()` 方法绘制饼图。
# `autopct='%1.1f%%'`：设置饼图中显示百分比的格式，这里是保留一位小数。
# `startangle=90`：设置饼图的起始角度为90度（即从顶部开始）。
# `colors=plt.cm.Paired.colors`：指定饼图扇区的颜色使用matplotlib的 'Paired' 颜色映射。
plt.title('Distribution of Exercise Frequency') # 设置饼图的标题。
plt.ylabel('')                              # 清空y轴标签，因为饼图通常不需要y轴标签。
plt.show()                                  # 显示绘制好的饼图。

import pandas as pd                        # 再次导入pandas（如果前面已经导入过，这里重复出现可能是分模块或阶段性编码的习惯）。
from sklearn.model_selection import train_test_split # 从scikit-learn库中导入train_test_split，一个用于分割数据集的工具。
import matplotlib.pyplot as plt            # 再次导入matplotlib.pyplot（同上）。

# 填充缺失值
data_filled = data.apply(lambda x: x.fillna(x.mode()[0]))
# 这一行代码是处理整个DataFrame的缺失值，通过“众数填充”的方式。
# `data.apply(...)`：对DataFrame的每一列（或行，取决于 `axis` 参数，默认为列）应用一个函数。
# `lambda x: x.fillna(x.mode()[0])`：这是一个匿名函数。
#   `x.mode()[0]`：计算当前列 `x` 的众数（出现次数最多的值）。`[0]` 是为了获取众数Series中的第一个值，因为可能存在多个众数。
#   `x.fillna(...)`：用计算出的众数来填充当前列 `x` 中的所有缺失值。
# 通过这种方式，所有列的缺失值都被相应列的众数填充。

# 划分数据（测试集占比20%）
train_data, test_data = train_test_split(data_filled,test_size=0.2, random_state=42)
# `train_test_split` 函数将 `data_filled` 分割成两个部分：训练集 (`train_data`) 和测试集 (`test_data`)。
# `test_size=0.2`：指定测试集占总数据集的比例是20%。
# `random_state=42`：这是一个随机种子。设置这个参数可以确保每次运行代码时，数据集的划分结果都是相同的，保证实验的可复现性。

# 保存处理后的数据
cleaned_file_path = '2.1.5_cleaned_data.csv' # 定义保存文件的路径和名称。
data_filled.to_csv(cleaned_file_path, index=False)
# `data_filled.to_csv(...)`：这是pandas用来将DataFrame保存为CSV文件的方法。
# `cleaned_file_path`：指定新保存的文件的路径和名称。
# `index=False`：这个参数非常重要，它表示在保存CSV文件时，不要把DataFrame的索引（左边默认从0开始的数字序号）也写入到CSV文件中。
# 这样做可以避免文件中有额外且无用的列。
